In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import torch
from torch.nn import BCELoss, CrossEntropyLoss, MSELoss


import os
DATA_PATH = os.path.join(os.path.dirname(os.getcwd()), 'data')
import torch.nn as nn

In [ ]:
"""
This is an example implementation of an LSTM agent. It has an input size of 3 and its output is size (64,)
"""

class LSTMAgent(nn.Module):
    def __init__(self):
        super(LSTMAgent, self).__init__()
        self.input_size = 3 # Vector for Rock Paper Sciscors
        self.hidden_size = 64 # Defining number of features

        # Create LSTM model
        self.lstm = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, batch_first=True)

        # Create final linear layer
        self.linear = nn.Linear(self.hidden_size, 1)

    def forward(self, x):
        # x has shape (batch_size, sequence_length, input_size)
        out, _ = self.lstm(x)
        out = self.linear(out[:, -1, :])
        return out.squeeze()


## Testing a basic LSTM implementation

In [ ]:
def data_to_num(df):

    agent_choice_dict = {"Paper": 0, "Rock": 1, "Scissors": 2}
    result_dict = {"W": 1, "L": -1, "D": 0}

    df["agents_choice"] = df["agents_choice"].apply(lambda x: agent_choice_dict[x])
    df["result"] = df["result"].apply(lambda x: result_dict[x])

    return df

In [ ]:
# data = torch.Tensor()
# t = torch.Tensor()

agent1_df = pd.read_csv(os.path.join(DATA_PATH, 'agent1_df_train.csv'), index_col=0)
agent2_df = pd.read_csv(os.path.join(DATA_PATH, 'agent2_df_train.csv'),index_col=0)
agent3_df = pd.read_csv(os.path.join(DATA_PATH, 'agent3_df_train.csv'),index_col=0)

print("Data before conversion:")
display(agent1_df.head(5))

agent1_df = data_to_num(agent1_df)
agent2_df = data_to_num(agent2_df)
agent3_df = data_to_num(agent3_df)

target1 = agent1_df.pop('result')
target2 = agent2_df.pop('result')
target3 = agent3_df.pop('result')

print("Data after conversion:")
display(agent1_df.head(5))


print("Target after conversion:")
display(target1.head(5))

In [ ]:
def get_sequences(df, seq_len=5):

    data = df.values # agent_choice
    
    data = np.array(data) == np.arange(3)

    input_seq = []
    target_seq = []

    for i in range(len(data) - seq_len):
        input_seq.append(data[i:i+seq_len])
        target_seq.append(data[i+seq_len, 0])

    return np.array(input_seq), np.array(target_seq)

input_seq, target_seq = get_sequences(agent1_df, seq_len=5)
print("Training Sequences: 1995, Window Size: 5, Number of Features: 1")
print(f"Input data shape: {input_seq.shape}")
print(f"Target data shape: {target_seq.shape}")
display(input_seq[:3], target_seq)

In [ ]:
model = LSTMAgent()
criterion = MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 100
data = torch.from_numpy(input_seq.astype("float32"))
t = torch.from_numpy(target_seq.astype("float32"))
batch_size = 16
num_baches = len(input_seq) // batch_size

for epoch in range(epochs):
    running_loss = 0
    for i in range(num_baches):
        inputs, targets = data[i*batch_size: (i+1)*batch_size], t[i*batch_size: (i+1)*batch_size]
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch: {epoch}, Loss: {running_loss/num_baches}")